# Workshop // Exploring Gender Bias in Word Embedding

## https://learn.responsibly.ai/word-embedding

Powerd by [`responsibly`](https://docs.responsibly.ai/) - Toolkit for auditing and mitigating bias and fairness of machine learning systems 🔎🤖🧰

# 💎 Part Nine: Have we really removed the bias?

Let's look on another metric, called **WEAT** (Word Embedding Association Test) which is inspired by **IAT** (Implicit-Association Test) from Pyschology.

**Caliskan, A., Bryson, J. J., & Narayanan, A. (2017). [Semantics derived automatically from language corpora contain human-like biases.](http://www.cs.bath.ac.uk/~jjb/ftp/CaliskanEtAl-authors-full.pdf) Science, 356(6334), 183-186.**


## 9.1 - Ingredients

1. Target words (e.g., Male ve. Female)

2. Attribute words (e.g., Math vs. Arts)

In [ ]:
from copy import deepcopy  # 🛠️ For copying a nested data structure in Python

from responsibly.we.weat import WEAT_DATA


# B. A. Nosek, M. R. Banaji, A. G. Greenwald, Math=male, me=female, therefore math≠me.,
# Journal of Personality and Social Psychology 83, 44 (2002).
weat_gender_science_arts = deepcopy(WEAT_DATA[7])

In [ ]:
# 🛠️ filter words from the original IAT experiment that are not presend in the reduced Word2Vec model

from responsibly.we.weat import _filter_by_model_weat_stimuli

_filter_by_model_weat_stimuli(weat_gender_science_arts, w2v_small)

In [ ]:
weat_gender_science_arts['first_attribute']

In [ ]:
weat_gender_science_arts['second_attribute']

In [ ]:
weat_gender_science_arts['first_target']

In [ ]:
weat_gender_science_arts['second_target']

## 9.2 - Recipe

➕ Male x Science

➖ Male x Arts

➖ Female x Science

➕ Female x Arts

In [ ]:
def calc_combination_similiarity(model, attribute, target):
    score = 0

    for attribute_word in attribute['words']:

        for target_word in target['words']:

            score += w2v_small.similarity(attribute_word,
                                          target_word)

    return score

In [ ]:
male_science_score = calc_combination_similiarity(w2v_small,
                                                  weat_gender_science_arts['first_attribute'],
                                                  weat_gender_science_arts['first_target'])

male_science_score

In [ ]:
male_arts_score = calc_combination_similiarity(w2v_small,
                                               weat_gender_science_arts['first_attribute'],
                                               weat_gender_science_arts['second_target'])

male_arts_score

In [ ]:
female_science_score = calc_combination_similiarity(w2v_small,
                                                    weat_gender_science_arts['second_attribute'],
                                                    weat_gender_science_arts['first_target'])

female_science_score

In [ ]:
female_arts_score = calc_combination_similiarity(w2v_small,
                                                 weat_gender_science_arts['second_attribute'],
                                                 weat_gender_science_arts['second_target'])

female_arts_score

In [ ]:
male_science_score - male_arts_score - female_science_score + female_arts_score

In [ ]:
len(weat_gender_science_arts['first_attribute']['words'])

In [ ]:
(male_science_score - male_arts_score - female_science_score + female_arts_score) / 8

## 9.3 - All WEAT Tests

In [ ]:
from responsibly.we import calc_all_weat

calc_all_weat(w2v_small, [weat_gender_science_arts])

### ⚡ Important Note: Our results are a bit different because we use a reduced Word2Vec.


### Results from the Paper (computed on the complete Word2Vec):

![](../images/weat-w2v.png)


### ⚡Caveats

#### Comparing WEAT to the IAT

- Individuals (IAT) vs. Words (WEAT)
- Therefore, the meanings of the effect size and p-value are totally different!

#### ⚡🦄 WEAT score definition

The definition of the WEAT score is structured differently (but it is computationally equivalent). The original formulation matters to compute the p-value. Refer to the paper for details.

### 🧪 Effect size comparision between human and machine bias

With the effect size, we can "compare" a human bias to a machine one. It raises the question whether the baseline for meauring bias/fairness of a machine should be human bias? Then a well-performing machine shouldn't be necessarily not biased, but only less biased than human (think about autonomous cars or semi-structured vs. unstructured interview).

## 9.4 - WEAT vs. IAT

Lewis, M., & Lupyan, G. [What are we learning from language? Associations between gender biases and distributional semantics in 25 languages](https://mollylewis.shinyapps.io/iatlang_SI/).

![](../images/iat-weat.png)


1. Implicit male-career association (adjusted for participant age, gender, and congruent/incongruent block order) as a function of the linguistic male-career association derived from word-embeddings *r*(23) = 0.48 [0.11, 0.74]; *p* = 0.01; *n* = 25; Study 1b). Each point corresponds to a language. The size of the point is proportional to the number of participants who come from the country in which the language is dominant (total = 656,636 participants). Linguistic associations are estimated from models trained on text in each language from the Wikipedia corpus. Larger values indicate a greater tendency to associate men with the concept of career and women with the concept of family.

2. Difference (UK minus US) in implicit association versus linguistic association for 31 IAT types (*N* = 27,045 participants). Error bands indicate standard error of the linear model estimate.


## 9.5 - Let's go back to our question - did we removed the bias?


Gonen, H., & Goldberg, Y. (2019, June). [Lipstick on a Pig: Debiasing Methods Cover up Systematic Gender Biases in Word Embeddings But do not Remove Them](https://arxiv.org/pdf/1903.03862.pdf). In Proceedings of the 2019 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies, Volume 1 (Long and Short Papers) (pp. 609-614).

They used multiple methods, we'll show only two:
1. WEAT
2. Neutral words clustering

In [ ]:
w2v_small_gender_bias.calc_direct_bias()

In [ ]:
w2v_small_gender_debias.calc_direct_bias()

### 9.5.1 -  WEAT - before and after

![](../images/weat-experiment.png)

See `responsibly` [demo page on word embedding](https://docs.responsibly.ail/notebooks/demo-word-embedding-bias.html#first-experiment-weat-before-and-after-debias) for a complete example.

### 9.5.2 - Clustering Neutral Gender Words

In [ ]:
w2v_vocab = {word for word in w2v_small_gender_bias.model.vocab.keys()}

# 🦄 how we got these words - read the Bolukbasi's paper for details
all_gender_specific_words = set(BOLUKBASI_DATA['gender']['specific_full_with_definitional_equalize'])

all_gender_neutral_words = w2v_vocab - all_gender_specific_words

print('#vocab =', len(w2v_vocab),
      '#specific =', len(all_gender_specific_words),
      '#neutral =', len(all_gender_neutral_words))

In [ ]:
neutral_words_gender_projections = [(w2v_small_gender_bias.project_on_direction(word), word)
                                    for word in all_gender_neutral_words]

neutral_words_gender_projections.sort()

In [ ]:
neutral_words_gender_projections[:-20:-1]

In [ ]:
neutral_words_gender_projections[:20]

In [ ]:
# Neutral words: top 500 male-biased and top 500 female-biased words

GenderBiasWE.plot_most_biased_clustering(w2v_small_gender_bias, w2v_small_gender_debias);

Note: In the paper, they got a stronger result, 92.5% accuracy for the debiased model.
However, they perform clustering on all the words from the reduced word embedding, both gender- neutral and specific words, and applied slightly different pre-processing.

### 9.5.3 - 💎 Strong words form the paper (emphasis mine):

> The experiments ...
reveal a **systematic bias** found in the embeddings,
which is **independent of the gender direction**.


> The implications are alarming: while suggested
debiasing methods work well at removing the gender direction, the **debiasing is mostly superficial**.
The bias stemming from world stereotypes and
learned from the corpus is **ingrained much more
deeply** in the embeddings space.


> .. real concern from biased representations is **not the association** of a concept with
words such as “he”, “she”, “boy”, “girl” **nor** being
able to perform **gender-stereotypical word analogies**... algorithmic discrimination is more likely to happen by associating one **implicitly gendered** term with
other implicitly gendered terms, or picking up on
**gender-specific regularities** in the corpus by learning to condition on gender-biased words, and generalizing to other gender-biased words.


# 💎💎 Part Ten: Meta "So What?" - II

## Can we debias at all a word embedding?

## Under some downstream use-cases, maybe the bias in the word embedding is desirable?